# 1. 라이브러리 및 패키지 불러오기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 2. 데이터 불러오기

In [2]:
log_data = pd.read_csv('/Users/sungahwang/Desktop/BC_data/log_data.csv')

In [3]:
relog = pd.read_csv('/Users/sungahwang/Desktop/BC_data/relog.csv')

In [4]:
relog = relog.drop(['Unnamed: 0'], axis = 1)

# 3. 전처리

##1) mp_os
- android os는 getcreditinfo행동밖에 없고, Android os는 getcreditinfo만 없으므로 합쳐주는게 맞다고 판단 android -> Android로 변환

- 애플 제품에는 login이 없음 / 아이패드에는 여윳돈 계산 기능이 없는것인지 하지 않은 것인지 암튼 없음

In [5]:
log_data.loc[list(log_data[log_data['mp_os']=='android'].index),'mp_os'] = 'Android'

In [6]:
log_data.loc[list(log_data[log_data['mp_os']=='iPadOS'].index),'mp_os'] = 'iOS'

In [7]:
# mp_os가 결측값이면 mp_app_version도 결측값임을 확인

# 4. 파생변수 추가

## 1) 각 유저가 하루에 행동을 얼마나 했는가?

In [8]:
log = log_data

In [9]:
grouped = log.groupby(['user_id', 'date_cd'])['user_id'].count()
df_grouped = pd.DataFrame(grouped)

In [10]:
df_grouped.rename(columns = {'user_id' : 'user_id_cnt'}, inplace = True)

In [11]:
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new = df_grouped_new.reset_index('user_id')

In [12]:
df_grouped_new.head()

,user_id,date_cd,user_id_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12


## 2) 유저가 했던 행동을 총합해서 하나의 컬럼으로 추출

In [13]:
sample = pd.DataFrame(df_grouped_new['user_id_cnt'].groupby(df_grouped_new['user_id']).sum())

In [14]:
log = pd.merge(relog , sample,
                 how = 'left',
                 on = 'user_id')
log

,user_id,timestamp,event,user_id_cnt
0,1,2022-05-03 14:52:28,11-11-5-7-11,5
1,7,2022-05-22 16:39:49,11,1
2,9,2022-05-21 23:37:58,11-11-11,3
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120
...,...,...,...,...
584631,879693,2022-05-13 11:29:49,1-2-8-3-4-1-3-4-5-11-1-3-4-1-2-8-1-2-8-1-3-3-3...,61
584632,879694,2022-03-31 20:07:23,1-2-8-3-3-3,6
584633,879695,2022-05-27 12:48:32,1-2-8-3,4
584634,879696,2022-03-14 05:35:34,7-5-11-11-5-11-4-7-2-8-3,11


## 3) 유저가 기간동안 총 몇일 접속했는지

In [15]:
grouped = log_data.groupby(['user_id', 'date_cd'])['user_id'].count()
df_grouped = pd.DataFrame(grouped)

In [16]:
df_grouped.rename(columns = {'user_id' : 'user_id_day_cnt'}, inplace = True)
df_grouped_new = df_grouped.reset_index('date_cd')
df_grouped_new = df_grouped_new.reset_index('user_id')
df_grouped_new.head()

,user_id,date_cd,user_id_day_cnt
0,1,2022-05-03,2
1,1,2022-06-16,3
2,7,2022-05-22,1
3,9,2022-05-21,3
4,11,2022-03-24,12


In [17]:
df_grouped_new['user_id_day_cnt'].groupby(df_grouped_new['user_id']).count()

user_id
1          2
7          1
9          1
11         4
12        14
          ..
879693     8
879694     1
879695     1
879696     1
879698     1
Name: user_id_day_cnt, Length: 584636, dtype: int64

In [18]:
sample1=pd.DataFrame(df_grouped_new['user_id_day_cnt'].groupby(df_grouped_new['user_id']).count())
sample1.head()

,user_id_day_cnt
user_id,
1,2
7,1
9,1
11,4
12,14


In [19]:
log = pd.merge(log , sample1,
                 how = 'left',
                 on = 'user_id')
log.head()

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt
0,1,2022-05-03 14:52:28,11-11-5-7-11,5,2
1,7,2022-05-22 16:39:49,11,1,1
2,9,2022-05-21 23:37:58,11-11-11,3,1
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43,4
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120,14


## 4) 한도 조회, 신용 정보 조회, 대출 관리 서비스 이용, 여윳돈 계산기 서비스 이용, DSR 계산기 서비스 이용 중 어떤 것을 먼저 이용한 사람인가에 따른 차이점

In [20]:
log1 = log_data

In [21]:
group = log1.groupby(['user_id', 'event'])['timestamp'].min()
df_group = pd.DataFrame(group)

In [22]:
df_group_new = df_group.reset_index()
df_final = df_group_new.drop_duplicates(['user_id'], keep = 'first')

In [23]:
df_final['event'].unique()

array(['GetCreditInfo', 'CompleteIDCertification', 'EndLoanApply',
       'Login', 'OpenApp', 'ViewLoanApplyIntro', 'StartLoanApply',
       'UseLoanManage', 'SignUp'], dtype=object)

In [24]:
df_final.loc[(df_final['event'] != 'GetCreditInfo') & (df_final['event'] != 'EndLoanApply')& 
             (df_final['event'] != 'UseLoanManage') & (df_final['event'] != 'ViewLoanApplyIntro')&
             (df_final['event'] != 'StartLoanApply') & (df_final['event'] != 'CompleteIDCertification'), 'event'] = 'others'

In [25]:
df_final = df_final.rename({'event':'first_event'}, axis = 1)
df_final

,user_id,first_event,timestamp
0,1,GetCreditInfo,2022-05-03 14:52:28
3,7,GetCreditInfo,2022-05-22 16:39:49
4,9,GetCreditInfo,2022-05-21 23:37:58
5,11,CompleteIDCertification,2022-03-24 10:56:26
13,12,CompleteIDCertification,2022-03-14 01:13:46
...,...,...,...
2860102,879693,CompleteIDCertification,2022-05-13 11:30:06
2860109,879694,CompleteIDCertification,2022-03-31 20:07:42
2860113,879695,CompleteIDCertification,2022-05-27 12:48:51
2860117,879696,CompleteIDCertification,2022-03-14 05:38:16


In [26]:
df_final = df_final.drop(['timestamp'], axis = 1)

In [27]:
log = pd.merge(log , df_final,
                 how = 'left',
                 on = 'user_id')
log.head()

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event
0,1,2022-05-03 14:52:28,11-11-5-7-11,5,2,GetCreditInfo
1,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo
2,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43,4,CompleteIDCertification
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120,14,CompleteIDCertification


# 5. Timeout 10분을 기준으로 세션 나누기

In [28]:
log2 = log_data
log2 = log2.sort_values(["user_id", "timestamp"], ascending= True)

In [29]:
log2['timestamp'] = pd.to_datetime(log2['timestamp'])
log2.head()

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
11709372,1,GetCreditInfo,2022-05-03 14:52:28,Android,464,2022-05-03
11709374,1,GetCreditInfo,2022-05-03 14:52:35,Android,464,2022-05-03
2451691,1,UseLoanManage,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
2451693,1,Login,2022-06-16 23:58:41,Android,3.12.1,2022-06-16
7071607,1,GetCreditInfo,2022-06-16 23:58:42,Android,464,2022-06-16


In [30]:
log2 = log2.drop(['mp_os', 'mp_app_version', 'date_cd'], axis = 1)

In [31]:
log2['timeout'] = log2.timestamp - log2.timestamp.shift()
log2.head()

,user_id,event,timestamp,timeout
11709372,1,GetCreditInfo,2022-05-03 14:52:28,NaT
11709374,1,GetCreditInfo,2022-05-03 14:52:35,0 days 00:00:07
2451691,1,UseLoanManage,2022-06-16 23:58:41,44 days 09:06:06
2451693,1,Login,2022-06-16 23:58:41,0 days 00:00:00
7071607,1,GetCreditInfo,2022-06-16 23:58:42,0 days 00:00:01


In [32]:
pastid = 0
userid = 0

# timeout의 기준을 10으로 함.
for i in range(len(log2)):
  userid = log2['user_id'].values[i]
  if userid != pastid:
    log2['timeout'].values[i] = 0
  if  log2['timeout'].values[i] > 10000000000:
    log2['timeout'].values[i] = 0
  pastid = userid

In [33]:
log2.head(8)

,user_id,event,timestamp,timeout
11709372,1,GetCreditInfo,2022-05-03 14:52:28,0 days 00:00:00
11709374,1,GetCreditInfo,2022-05-03 14:52:35,0 days 00:00:07
2451691,1,UseLoanManage,2022-06-16 23:58:41,0 days 00:00:00
2451693,1,Login,2022-06-16 23:58:41,0 days 00:00:00
7071607,1,GetCreditInfo,2022-06-16 23:58:42,0 days 00:00:01
10428909,7,GetCreditInfo,2022-05-22 16:39:49,0 days 00:00:00
9627339,9,GetCreditInfo,2022-05-21 23:37:58,0 days 00:00:00
9627368,9,GetCreditInfo,2022-05-21 23:43:33,0 days 00:00:00


In [34]:
grouped = log2.groupby(['user_id'])['timeout'].sum()
df_grouped = pd.DataFrame(grouped)

In [35]:
df_grouped

,timeout
user_id,
1,0 days 00:00:08
7,0 days 00:00:00
9,0 days 00:00:00
11,0 days 00:00:30
12,0 days 00:03:20
...,...
879693,0 days 00:01:26
879694,0 days 00:00:00
879695,0 days 00:00:00


In [36]:
df_grouped.rename(columns = {'timeout' : 'timeout_sum'}, inplace = True)

In [37]:
log = pd.merge(log , df_grouped,
                 how = 'left',
                 on = 'user_id')
log.head()

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event,timeout_sum
0,1,2022-05-03 14:52:28,11-11-5-7-11,5,2,GetCreditInfo,0 days 00:00:08
1,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo,0 days 00:00:00
2,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo,0 days 00:00:00
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43,4,CompleteIDCertification,0 days 00:00:30
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120,14,CompleteIDCertification,0 days 00:03:20


## timeout 초 단위

In [38]:
log['timeout_sum(s)'] = pd.to_timedelta(log['timeout_sum']).view(np.int64) / 1e9
log = log.drop(['timeout_sum'], axis = 1)

In [39]:
log

,user_id,timestamp,event,user_id_cnt,user_id_day_cnt,first_event,timeout_sum(s)
0,1,2022-05-03 14:52:28,11-11-5-7-11,5,2,GetCreditInfo,8.0
1,7,2022-05-22 16:39:49,11,1,1,GetCreditInfo,0.0
2,9,2022-05-21 23:37:58,11-11-11,3,1,GetCreditInfo,0.0
3,11,2022-03-24 10:53:59,4-11-5-11-6-1-2-8-3-5-5-11-4-11-5-2-8-3-11-5-5...,43,4,CompleteIDCertification,30.0
4,12,2022-03-14 01:13:11,4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-3-4-7-1-2-8-3-3-...,120,14,CompleteIDCertification,200.0
...,...,...,...,...,...,...,...
584631,879693,2022-05-13 11:29:49,1-2-8-3-4-1-3-4-5-11-1-3-4-1-2-8-1-2-8-1-3-3-3...,61,8,CompleteIDCertification,86.0
584632,879694,2022-03-31 20:07:23,1-2-8-3-3-3,6,1,CompleteIDCertification,0.0
584633,879695,2022-05-27 12:48:32,1-2-8-3,4,1,CompleteIDCertification,0.0
584634,879696,2022-03-14 05:35:34,7-5-11-11-5-11-4-7-2-8-3,11,1,CompleteIDCertification,30.0


## 7. Log+Spec+Loan

In [40]:
join = pd.read_csv('/Users/sungahwang/Desktop/BC_data/Preprocessing+loan.csv')

In [41]:
join = join.rename({'Labels':'spec_clust'}, axis = 1)

In [42]:
log.rename(columns={'user_id_cnt': 'action_cnt'}, inplace=True)

In [43]:
log.rename(columns={'user_id_day_cnt': 'use_day_cnt'}, inplace=True)

In [44]:
# first_event 결측값 -> Others
log['first_event'] = log['first_event'].fillna('Others')

In [45]:
join_final = pd.merge(join,log ,on="user_id",how="left")

## 전처리

In [46]:
# id 당 총 대출 수
id_applied = join_final.groupby('user_id').sum()['is_applied'].sort_values(ascending=False)
id_applied = pd.DataFrame(id_applied)
id_applied = id_applied.reset_index()
id_applied = id_applied.rename({'is_applied':'sum_applied'}, axis = 1)

In [47]:
join = pd.merge(id_applied,join_final ,on="user_id",how="right")

In [48]:
sum_applied_list = id_applied['sum_applied'].unique()

## 8. 결측값

### Join으로 인하여 생기는 결측값 중 is_applied가 0인 데이터는 제거

In [49]:
join.isnull().sum()

user_id                                      0
sum_applied                                  0
application_id                               0
gender                                       0
insert_time                                  0
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
reage                                        0
career                                       0
credit_grade                                 0
spec_clust                                   0
loanapply_insert_time                        0
bank_id                                      0
product_id   

In [50]:
join_0 = join[join['is_applied']==0]
join_not0 = join[join['is_applied']!=0]

In [51]:
join_0 = join_0.dropna(subset = ['timestamp'], axis=0) 

In [52]:
join = pd.concat([join_0,join_not0])

In [53]:
join = join.drop(['timestamp'], axis = 1)

In [54]:
join = join.drop(['event'], axis = 1)

In [55]:
from pandas import Series, DataFrame

In [56]:
for i in range(len(sum_applied_list)):
  join.loc[join['action_cnt'].isnull() & join['sum_applied']== sum_applied_list[i], 'action_cnt'] = join[(join['sum_applied']==sum_applied_list[i])]['action_cnt'].mean()
  join.loc[join['use_day_cnt'].isnull() & join['sum_applied']== sum_applied_list[i], 'use_day_cnt'] = join[(join['sum_applied']==sum_applied_list[i])]['use_day_cnt'].mean()
  join.loc[join['timeout_sum(s)'].isnull() & join['sum_applied']== sum_applied_list[i], 'timeout_sum(s)'] = join[(join['sum_applied']==sum_applied_list[i])]['timeout_sum(s)'].mean()

In [57]:
df_max = join.groupby(['cluster2', 'first_event'])['first_event'].value_counts()
df_max

cluster2  first_event              first_event            
0         CompleteIDCertification  CompleteIDCertification    9276934
          EndLoanApply             EndLoanApply                257282
          GetCreditInfo            GetCreditInfo               575553
          StartLoanApply           StartLoanApply                   3
          others                   others                       42723
1         CompleteIDCertification  CompleteIDCertification    1108779
          EndLoanApply             EndLoanApply                 27751
          GetCreditInfo            GetCreditInfo               100534
          others                   others                         784
2         CompleteIDCertification  CompleteIDCertification     303297
          EndLoanApply             EndLoanApply                  7416
          GetCreditInfo            GetCreditInfo                14192
          others                   others                        1068
3         CompleteIDCertificati

In [58]:
join.loc[join['first_event'].isnull(), 'first_event'] = 'CompleteIDCertification'

In [59]:
join.isnull().sum()

user_id                                      0
sum_applied                                  0
application_id                               0
gender                                       0
insert_time                                  0
yearly_income                                0
income_type                                  0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn                   0
personal_rehabilitation_complete_yn          0
existing_loan_cnt                            0
existing_loan_amt                            0
reage                                        0
career                                       0
credit_grade                                 0
spec_clust                                   0
loanapply_insert_time                        0
bank_id                                      0
product_id   

In [60]:
join.to_csv('/Users/sungahwang/Desktop/BC_data/final.csv', index = False)